# DuyKhanh Nguyen
300316520

# Shringar 
3003356399

# Spam Classification Dectection - CSIS 4260 Project

# Referrences

# A. Prepare Data

In [1]:
# Import all the require important library
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.feature_selection import SelectFromModel

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool

from sklearn.preprocessing import StandardScaler

import itertools

from sklearn.pipeline import Pipeline

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report



In [2]:
df = pd.read_csv('Datasets/emails.csv')
df


,Email No.,the,to,ect,and,for,of,a,you,hou,...,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction
0,Email 1,0,0,1,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Email 2,8,13,24,6,6,2,102,1,27,...,0,0,0,0,0,0,0,1,0,0
2,Email 3,0,0,1,0,0,0,8,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Email 4,0,5,22,0,5,1,51,2,10,...,0,0,0,0,0,0,0,0,0,0
4,Email 5,7,6,17,1,5,2,57,0,9,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5167,Email 5168,2,2,2,3,0,0,32,0,0,...,0,0,0,0,0,0,0,0,0,0
5168,Email 5169,35,27,11,2,6,5,151,4,3,...,0,0,0,0,0,0,0,1,0,0
5169,Email 5170,0,0,1,1,0,0,11,0,0,...,0,0,0,0,0,0,0,0,0,1
5170,Email 5171,2,7,1,0,2,1,28,2,0,...,0,0,0,0,0,0,0,1,0,1


In [3]:
# Drop the uneeded columns
df = df.drop("Email No.", axis = 1)

In [4]:
# Check null values
df.isnull().sum().sum()

0

### Note: The dataset is already clean, we can focus more on the models and analysis

# B. Classification models

In [5]:
# Split Data
target = df["Prediction"]
features = df.drop("Prediction", axis = 1)

X_train, X_test, Y_train, Y_test = train_test_split(features, target, test_size = 0.25, random_state = 42)

### Step 1: Features Selection

In [6]:
# SelectFromModel feature selection with RandomForest Classifier as the estimator. You can use the number of estimators between 100 to 200
sel = SelectFromModel(estimator = RandomForestClassifier(n_estimators = 150))

### Step 2: Scaling

In [7]:
# Scaler. I will choose StandardScaler
scaler = StandardScaler()

### Step 3: Buidling Models and its hyperparameters

In [8]:
# Create a List to contains all the models
models_list = []

# Logistict Regression Hyperparameters
Cs = [0.0001, 1, 1000]
multi_classes = ['auto', 'ovr', 'multinomial']
hyperparams = list(itertools.product(Cs, multi_classes))
# Logsitic Regression bulding models
for C, multi_class in hyperparams:
    model = LogisticRegression(n_jobs= -1, C = C, multi_class = multi_class)
    model_name = "Logistic Regression with C = {}, and multi_class = {}".format(C, multi_class) 
    models_list.append([model_name, model])

# KNN Hyperparameters
leaf_sizes = list(range(1, 3))
n_neighbors = list(range(1, 3))
hyperparams = list(itertools.product(leaf_sizes, n_neighbors))
# KNN building models
for leaf_size, n_neighbor in hyperparams:
    model = KNeighborsClassifier(n_jobs= -1, leaf_size = leaf_size, n_neighbors = n_neighbor)
    model_name = "KNN with leaf_size = {}, and n_neighbor = {}".format(leaf_size, n_neighbor) 
    models_list.append([model_name, model])


# SVC Hyperparameters
gammas = [0.1, 1, 10]
Cs = [0.0001, 1, 1000]
hyperparams = list(itertools.product(gammas, Cs))
# Linear SVC
for gamma, C in hyperparams:
    model = SVC(kernel = "linear", gamma = gamma, C = C)
    model_name = "Linear SVC with gamma = {}, and C = {}".format(gamma, C) 
    models_list.append([model_name, model])
# RBF SVC
for gamma, C in hyperparams:
    model = SVC(kernel = "rbf", gamma = gamma, C = C)
    model_name = "RBF SVC with gamma = {}, and C = {}".format(gamma, C) 
    models_list.append([model_name, model])

# Decision Tree Hyperparameters
criterions = ["gini", "entropy", "log_loss"]
splitters = ["best", "random"]
hyperparams = list(itertools.product(criterions, splitters))
for criterion, splitter in hyperparams:
    model = DecisionTreeClassifier(criterion = criterion, splitter = splitter)
    model_name = "Decision Tree with criterion = {}, and splitter = {}".format(criterion, splitter) 
    models_list.append([model_name, model])
    
# Naive Bayes Hyperparameters
var_smoothings = [1e-10, 1e-8, 1e-6, 1e-4, 1e-2]
for var_smoothing in var_smoothings:
    model = GaussianNB(var_smoothing = var_smoothing)
    model_name = "Naive Bayes with var_smoothing = {}".format(var_smoothing) 
    models_list.append([model_name, model])
    
# Random Forest Hyperparameters
criterions = ["gini", "entropy", "log_loss"]
n_estimators = [100, 150, 200]
hyperparams = list(itertools.product(criterions, n_estimators))
for criterion, n_estimator in hyperparams:
    model = RandomForestClassifier(n_jobs = -1, criterion = criterion, n_estimators = n_estimator)
    model_name = "RandomForestClassifier with criterion = {}, and n_estimators = {}".format(criterion, n_estimator) 
    models_list.append([model_name, model])
    
# AdaBoost Hyperparameters
learning_rates = [0.1, 1, 10]
n_estimators = [100, 150, 200]
hyperparams = list(itertools.product(learning_rates, n_estimators))
for learning_rate, n_estimator in hyperparams:
    model = AdaBoostClassifier(learning_rate = learning_rate, n_estimators = n_estimator)
    model_name = "AdaBoostClassifier with learning_rate = {}, and n_estimators = {}".format(learning_rate, n_estimator) 
    models_list.append([model_name, model])

# XGBoost Hyperparameters
learning_rates = [0.1, 0.5, 1]
n_estimators = [100, 150, 200]
hyperparams = list(itertools.product(learning_rates, n_estimators))
for learning_rate, n_estimator in hyperparams:
    model = XGBClassifier(n_jobs = -1, learning_rate = learning_rate, n_estimators = n_estimator, use_label_encoder = False)
    model_name = "XGBClassifier with criterion = {}, and n_estimators = {}".format(learning_rate, n_estimator) 
    models_list.append([model_name, model])

# CatBoost Hyperparameters
learning_rates = [0.1, 0.5, 1]
iterationses = [5, 10, 20]
hyperparams = list(itertools.product(learning_rates, iterationses))
for learning_rate, iterations in hyperparams:
    model = CatBoostClassifier(learning_rate = learning_rate, iterations = iterations)
    model_name = "CatBoostClassifier with criterion = {}, and iterations = {}".format(learning_rate, iterations) 
    models_list.append([model_name, model])


### Step 4: Create a pipeline for feature selectionm, scaler, and each of all models with its hyperparameters

In [9]:
# Create a pipeline for feature selectionm scaler, and each of all models with its hyperparameters
scores = []
names = []
pipe_list = []
for model in models_list:
    pipe = Pipeline([("Scaler", scaler),
                     ("Features Selection", sel),
                     ("Classifier", model[1])])
    pipe.fit(X_train, Y_train)
    pipe_list.append(pipe)
    score = pipe.score(X_test, Y_test)
    scores.append(score)
    name = model[0]
    names.append(name)
    

[11:04:54] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:04:57] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:05:01] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:05:05] WARNING:

### Step 5: Sorting the score result and choose the best model

In [10]:
scores_df = pd.DataFrame(zip(names, scores), columns = ["Classifier", "Accuracy"])
print(scores_df.sort_values(by= "Accuracy", ascending = False).to_markdown())

|    | Classifier                                                               |   Accuracy |
|---:|:-------------------------------------------------------------------------|-----------:|
| 62 | XGBClassifier with criterion = 0.1, and n_estimators = 200               |   0.977572 |
| 61 | XGBClassifier with criterion = 0.1, and n_estimators = 150               |   0.976798 |
| 65 | XGBClassifier with criterion = 0.5, and n_estimators = 200               |   0.976025 |
| 50 | RandomForestClassifier with criterion = log_loss, and n_estimators = 200 |   0.976025 |
| 47 | RandomForestClassifier with criterion = entropy, and n_estimators = 200  |   0.976025 |
| 43 | RandomForestClassifier with criterion = gini, and n_estimators = 150     |   0.976025 |
| 46 | RandomForestClassifier with criterion = entropy, and n_estimators = 150  |   0.975251 |
| 64 | XGBClassifier with criterion = 0.5, and n_estimators = 150               |   0.974478 |
| 63 | XGBClassifier with criterion = 0.5, and n_e